In [45]:
## Import Packages
import numpy as np
import scipy.stats as stats
import os
import pickle
import scipy.io as scio
from scipy.interpolate import griddata
from ncdump_python3 import ncdump
import pickle
import Area_Avg
import Jacks_Functions as JF
from netCDF4 import Dataset
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

In [73]:
#Load in Dimensions
Source_File = Dataset('Kernels/Interpolated/CAM3_Kernels.nc')

#CAM3 Horizontal resolution grid
CAM3_Lat = np.squeeze(Source_File.variables['lat'])
CAM3_Lon = np.squeeze(Source_File.variables['lon'])

LLL = Dataset('LatLon.nc')
WACCM4_Lat = np.squeeze(LLL.variables['lat'])
WACCM4_Lon = np.squeeze(LLL.variables['lon'])
LLL.close()

#Lists for iterating
Months = np.arange(0,12)
FP_Ensemble_Members = np.arange(0,5)

In [100]:
#load cloud radiative effects

wAVD_dFLUT_CAM3Grid = np.swapaxes(pickle.load(open(\
    "Future Projection CC Responses/wAVD_dFLUT_CAM3Grid.pickle","rb"),encoding='latin1'),2,3)
wAVD_dFLUTC_CAM3Grid = np.swapaxes(pickle.load(open(\
    "Future Projection CC Responses/wAVD_dFLUTC_CAM3Grid.pickle","rb"),encoding='latin1'),2,3)
wAVD_dFSNT_CAM3Grid = np.swapaxes(pickle.load(open(\
    "Future Projection CC Responses/wAVD_dFSNT_CAM3Grid.pickle","rb"),encoding='latin1'),2,3)
wAVD_dFSNTC_CAM3Grid = np.swapaxes(pickle.load(open(\
    "Future Projection CC Responses/wAVD_dFSNTC_CAM3Grid.pickle","rb"),encoding='latin1'),2,3)

#Cloud radiative effect
wAVD_dCRE_SW_CAM3Grid = wAVD_dFSNTC_CAM3Grid-wAVD_dFSNT_CAM3Grid
wAVD_dCRE_LW_CAM3Grid = wAVD_dFLUTC_CAM3Grid-wAVD_dFLUT_CAM3Grid

In [101]:
# load forcing corrections
wAVD_dLW_RF_CAM3Grid = pickle.load(open(\
"ENERGY BUDGET/Cloud/wAVD/wAVD_RF_LW_CAM3Grid.pickle","rb"))
wAVD_dLW_RF_clear_CAM3Grid = pickle.load(open(\
"ENERGY BUDGET/Cloud/wAVD/wAVD_RF_LW_clear_CAM3Grid.pickle","rb"))
CAM3_wAVD_RF_LW_Correction = wAVD_dLW_RF_clear_CAM3Grid-wAVD_dLW_RF_CAM3Grid

wAVD_dSW_RF_CAM3Grid = pickle.load(open(\
"ENERGY BUDGET/Cloud/wAVD/wAVD_RF_SW_CAM3Grid.pickle","rb"))
wAVD_dSW_RF_clear_CAM3Grid = pickle.load(open(\
"ENERGY BUDGET/Cloud/wAVD/wAVD_RF_SW_clear_CAM3Grid.pickle","rb"))
CAM3_wAVD_RF_SW_Correction = wAVD_dSW_RF_clear_CAM3Grid-wAVD_dSW_RF_CAM3Grid

In [102]:
#load kernels

#albedo
Kernel_Alb = np.ma.masked_greater(np.squeeze(Source_File.variables['Alb_TOA']),1e10)
Kernel_Alb_clr = np.ma.masked_greater(np.squeeze(Source_File.variables['Alb_TOA_CLR']),1e10)

#Surface Temp
Kernel_Ts = np.ma.masked_greater(np.squeeze(Source_File.variables['Ts_TOA']),1e10)
Kernel_Ts_clr = np.ma.masked_greater(np.squeeze(Source_File.variables['Ts_TOA_CLR']),1e10)

#Air Temp
Kernel_Ta = np.ma.masked_greater(np.squeeze(Source_File.variables['Ta_TOA']),1e10)
Kernel_Ta_clr = np.ma.masked_greater(np.squeeze(Source_File.variables['Ta_TOA_CLR']),1e10)

#water vapour
Kernel_Qlw = np.ma.masked_greater(np.squeeze(Source_File.variables['WVlw_TOA']),1e10)
Kernel_Qlw_clr = np.ma.masked_greater(np.squeeze(Source_File.variables['WVlw_TOA_CLR']),1e10)

Kernel_Qsw = np.ma.masked_greater(np.squeeze(Source_File.variables['WVsw_TOA']),1e10)
Kernel_Qsw_clr = np.ma.masked_greater(np.squeeze(Source_File.variables['WVsw_TOA_CLR']),1e10)

In [103]:
#load Albedo responses, Calculate Correctional feedback

wAVD_dALB_CAM3Grid = np.swapaxes(pickle.load(open(\
    "Future Projection CC Responses/wAVD_dALB_CAM3Grid.pickle","rb"),encoding='latin1'),2,3)
wAVD_dALB_CAM3Grid = np.ma.masked_invalid(wAVD_dALB_CAM3Grid)*100

CAM3_wAVD_ALB_dSW = np.ma.zeros([5,12,64,128])
CAM3_wAVD_ALB_dSW_Clear = np.ma.zeros([5,12,64,128])

for i in range(5):    
    CAM3_wAVD_ALB_dSW[i,:,:,:] = np.ma.multiply(\
    wAVD_dALB_CAM3Grid[i,:,:,:],Kernel_Alb)
    
    CAM3_wAVD_ALB_dSW_Clear[i,:,:,:] = np.ma.multiply(\
    wAVD_dALB_CAM3Grid[i,:,:,:],Kernel_Alb_clr)
    
CAM3_wAVD_ALB_Correction = CAM3_wAVD_ALB_dSW_Clear-CAM3_wAVD_ALB_dSW

In [104]:
cx,cy = np.meshgrid(CAM3_Lon,CAM3_Lat)
wx,wy = np.meshgrid(WACCM4_Lon,WACCM4_Lat)

wAVD_PS = pickle.load(open('wAVD_Base_PS.pickle','rb'))
#interpolate the surface pressure to the CAM3 grid
wAVD_PS_interp = np.zeros([12,64,128])
for i in range(12):
    wAVD_PS_interp[i,:,:] = griddata(\
    (wx.flatten(),wy.flatten()),wAVD_PS[i,:,:].flatten(),\
    (cx,cy),method='linear')/100 #swap to hpa in the process

In [105]:
#Tile the plevs to match the shape of the kernel
CMIP_plevs_scalar = np.squeeze(Source_File.variables['lev'])

A = np.tile(CMIP_plevs_scalar.T,(CAM3_Lon.size,1))
B = np.transpose(A[:,:,None],(0,1,2))
C = np.tile(B,(1,1,CAM3_Lat.size))
D = np.transpose(C[:,:,:,None],(0,1,2,3))
E = np.tile(D,(1,1,1,12))
CMIP_plevs = np.swapaxes(E,3,0)
CMIP_plevs.shape

(12, 17, 64, 128)

In [106]:
#Calculate pressure level thickness using CMIP pressure levels and surface pressure
dp = np.zeros([12,17,64,128])

for i in range(12):
    for j in range(64):
        for k in range(128):
            dp[i,:,j,k] = JF.PlevThck(\
            PS = wAVD_PS_interp[i,j,k],plevs=CMIP_plevs_scalar,\
            p_top = min(CMIP_plevs_scalar))
            
dp = np.ma.masked_invalid(dp[:,:,:,:]/100) #CAM3 Kernel units are per 100 hPa

In [107]:
#estimation of the tropopause. 300hPA at the poles, raising
    #linearly to 100hpa at the tropics

p_tropopause_zonalmean_linear = np.zeros([64])

for y in range(64):
    if y <= len(CAM3_Lat)/2:
        if y == 0:
            p_tropopause_zonalmean_linear[y] = 300
        elif y == len(CAM3_Lat)/2:
            p_tropopause_zonalmean_linear[y] = 100
        else:
            p_tropopause_zonalmean_linear[y] = \
            p_tropopause_zonalmean_linear[y-1]\
            -(200/(len(CAM3_Lat)/2-1))
    else:
        if y == len(CAM3_Lat)/2:
            p_tropopause_zonalmean_linear[y] = 100
        elif y == len(CAM3_Lat):
            p_tropopause_zonalmean_linear[y] = 300
        else:
            p_tropopause_zonalmean_linear[y] = \
            p_tropopause_zonalmean_linear[y-1]\
            +(200/(len(CAM3_Lat)/2-1))
            
Q = np.tile(p_tropopause_zonalmean_linear.T,(CAM3_Lon.size,1))
R = np.transpose(Q[:,:,None],(0,1,2))
S = np.tile(R,(1,1,17))
T = np.transpose(S[:,:,:,None],(0,1,2,3))
U = np.tile(T,(1,1,1,12))
V = np.swapaxes(U,3,0)
p_tropopause_CAM3 = np.swapaxes(V,2,1)

#how does it look?
p_tropopause_CAM3.shape

(12, 17, 64, 128)

In [108]:
#climate change responses for temperature
wAVD_dST_CAM3Grid = np.swapaxes(pickle.load(open(\
   "Future Projection CC Responses/wAVD_dST_CAM3Grid.pickle","rb"),encoding='latin1'),2,3)
wAVD_dAT_CAM3Grid = np.swapaxes(pickle.load(open(\
    "Future Projection CC Responses/wAVD_dAT_CAM3Grid.pickle","rb"),encoding='latin1'),3,4)

In [109]:
CAM3_wAVD_Ts_dLW = np.zeros([5,12,64,128])
CAM3_wAVD_Ts_dLW_Clear = np.zeros([5,12,64,128])
CAM3_wAVD_dTa_Tropo = np.zeros([5,12,17,64,128])
CAM3_wAVD_Ta_dLW = np.zeros([5,12,64,128])
CAM3_wAVD_Ta_dLW_Clear = np.zeros([5,12,64,128])

for i in range(5):
    CAM3_wAVD_Ts_dLW[i,:,:,:] = Kernel_Ts*wAVD_dST_CAM3Grid[i,:,:,:]
    CAM3_wAVD_Ts_dLW_Clear[i,:,:,:] = Kernel_Ts_clr*wAVD_dST_CAM3Grid[i,:,:,:]
    
    CAM3_wAVD_dTa_Tropo[i,:,:,:,:] = \
    wAVD_dAT_CAM3Grid[i,:,:,:,:]*(CMIP_plevs>=p_tropopause_CAM3)
    
    CAM3_wAVD_Ta_dLW[i,:,:,:] = \
    np.ma.sum(Kernel_Ta*CAM3_wAVD_dTa_Tropo[i,:,:,:,:]*dp,1)
    
    CAM3_wAVD_Ta_dLW_Clear[i,:,:,:] = \
    np.ma.sum(Kernel_Ta_clr*CAM3_wAVD_dTa_Tropo[i,:,:,:,:]*dp,1)
    
CAM3_wAVD_Ts_Correction = CAM3_wAVD_Ts_dLW_Clear-CAM3_wAVD_Ts_dLW
CAM3_wAVD_Ta_Correction = CAM3_wAVD_Ta_dLW_Clear-CAM3_wAVD_Ta_dLW

In [110]:
#load base/response variables for saturation mixing ratio calculations
wAVD_Q_Base = np.swapaxes(pickle.load(open(\
   "Vars for Specific Humidity Calculations/wAVD_Q_2006-2015_CAM3Grid.pickle","rb")\
                                       ,encoding='latin1'),3,4)

wAVD_Q_Response = np.swapaxes(pickle.load(open(\
   "Vars for Specific Humidity Calculations/wAVD_Q_2056-2065_CAM3Grid.pickle","rb")\
                                       ,encoding='latin1'),3,4)

wAVD_Ta_Base = np.swapaxes(pickle.load(open(\
   "Vars for Specific Humidity Calculations/wAVD_Ta_2006-2015_CAM3Grid.pickle","rb")\
                                       ,encoding='latin1'),3,4)

wAVD_Ta_Response = np.swapaxes(pickle.load(open(\
   "Vars for Specific Humidity Calculations/wAVD_Ta_2056-2065_CAM3Grid.pickle","rb")\
                                       ,encoding='latin1'),3,4)

In [111]:
wAVD_sHum_Base = np.zeros([5,12,17,64,128])
wAVD_sHum_Response = np.zeros([5,12,17,64,128])

for i in range(FP_Ensemble_Members.size):
    #saturation mixing ratio for the baseline period
    wAVD_sHum_Base[i,:,:,:,:] = JF.Calc_SatSpec_Hum(\
    Ta=wAVD_Ta_Base[i,:,:,:,:],P = CMIP_plevs)/1000 #in kg/kg
    
    #saturation mixing ratio for the response period
    wAVD_sHum_Response[i,:,:,:,:] = JF.Calc_SatSpec_Hum(\
    Ta=wAVD_Ta_Response[i,:,:,:,:],P = CMIP_plevs)/1000 #in kg/kg
    
    
#relative humidity of the baseline, in kg/kg
wAVD_rh = wAVD_Q_Base/wAVD_sHum_Base
    
#new change in saturation spec hum under a +1k climate
wAVD_dQs1k = (wAVD_sHum_Response-wAVD_sHum_Base)/(wAVD_Ta_Response-wAVD_Ta_Base)

#change in spec hum assuming relative humidity remains constant
wAVD_dQ1k = wAVD_dQs1k*wAVD_rh

#logarithm of spec humidity required to increase temp by 1K under constant relative humidity
wAVD_dlnQ1k = wAVD_dQ1k/wAVD_Q_Base

In [112]:
#water vapour mixing ratios
#dln(specific humidity) in kg/kg
wAVD_dlnQ_CAM3Grid = np.swapaxes(pickle.load(open(\
   "Future Projection CC Responses/wAVD_dlnQ_CAM3Grid.pickle","rb"),encoding='latin1'),3,4)

#normalize the dln(specific humidity) by the amount required to increase temperature by 1K
    #(by the same anomaly used to compute the kernel)
wAVD_dlnQ_CAM3Grid_Norm = wAVD_dlnQ_CAM3Grid/wAVD_dlnQ1k

In [113]:
CAM3_wAVD_dlnQ_tropo = np.zeros([5,12,17,64,128])
CAM3_wAVD_Q_dLW = np.zeros([5,12,64,128])
CAM3_wAVD_Q_dLW_Clear = np.zeros([5,12,64,128])
CAM3_wAVD_Q_dSW = np.zeros([5,12,64,128])
CAM3_wAVD_Q_dSW_Clear = np.zeros([5,12,64,128])

for i in range(5):
    CAM3_wAVD_dlnQ_tropo[i,:,:,:,:] = \
    wAVD_dlnQ_CAM3Grid_Norm[i,:,:,:,:]*(CMIP_plevs>=p_tropopause_CAM3)
    
    CAM3_wAVD_Q_dLW[i,:,:,:] = \
    np.ma.sum(CAM3_wAVD_dlnQ_tropo[i,:,:,:,:]*Kernel_Qlw*dp,axis=1)
    
    CAM3_wAVD_Q_dLW_Clear[i,:,:,:] = \
    np.ma.sum(CAM3_wAVD_dlnQ_tropo[i,:,:,:,:]*Kernel_Qlw_clr*dp,axis=1)
    
    CAM3_wAVD_Q_dSW[i,:,:,:] = \
    np.ma.sum(CAM3_wAVD_dlnQ_tropo[i,:,:,:,:]*Kernel_Qsw*dp,axis=1)
    
    CAM3_wAVD_Q_dSW_Clear[i,:,:,:] = \
    np.ma.sum(CAM3_wAVD_dlnQ_tropo[i,:,:,:,:]*Kernel_Qsw_clr*dp,axis=1)
    
CAM3_wAVD_Qlw_Correction = CAM3_wAVD_Q_dLW_Clear-CAM3_wAVD_Q_dLW
CAM3_wAVD_Qsw_Correction = CAM3_wAVD_Q_dSW_Clear-CAM3_wAVD_Q_dSW

In [114]:
#Cloud Feedback Calculation
#orientation is tricky here. The fluxes calculated with kernels, port, and the GCM have
    #different sign conventions
CAM3_wAVD_CLD_LW_EnergyB = wAVD_dCRE_LW_CAM3Grid+\
                            CAM3_wAVD_RF_LW_Correction+\
                            CAM3_wAVD_Ta_Correction+\
                            CAM3_wAVD_Ts_Correction+\
                            CAM3_wAVD_Qlw_Correction

CAM3_wAVD_CLD_SW_EnergyB = -wAVD_dCRE_SW_CAM3Grid+\
                            CAM3_wAVD_RF_SW_Correction+\
                            CAM3_wAVD_ALB_Correction+\
                            CAM3_wAVD_Qsw_Correction

In [115]:
CAM3_wAVD_CLD_LW_EnergyB_AA = np.zeros([5,12])
CAM3_wAVD_CLD_LW_EnergyB_GA = np.zeros([5,12])
CAM3_wAVD_CLD_SW_EnergyB_AA = np.zeros([5,12])
CAM3_wAVD_CLD_SW_EnergyB_GA = np.zeros([5,12])

for i in range(5):
    CAM3_wAVD_CLD_LW_EnergyB_AA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_wAVD_CLD_LW_EnergyB[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    CAM3_wAVD_CLD_LW_EnergyB_GA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_wAVD_CLD_LW_EnergyB[i,:,:,:],CAM3_Lat,CAM3_Lon)
    
    CAM3_wAVD_CLD_SW_EnergyB_AA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_wAVD_CLD_SW_EnergyB[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    CAM3_wAVD_CLD_SW_EnergyB_GA[i,:] = Area_Avg.LatLonavg_Time(\
    CAM3_wAVD_CLD_SW_EnergyB[i,:,:,:],CAM3_Lat,CAM3_Lon)

In [116]:
print(np.mean(CAM3_wAVD_CLD_SW_EnergyB_AA,axis=1))
print(np.mean(CAM3_wAVD_CLD_LW_EnergyB_AA,axis=1))

[-0.37108103 -0.28715281 -0.46702203 -0.90702493 -0.44123428]
[1.34064494 1.18885293 1.45742019 1.66062788 1.54626383]


In [91]:
wAVD_dST_CAM3Grid_AA = np.zeros([5,12])
wAVD_dST_CAM3Grid_GA = np.zeros([5,12])

for i in range(FP_Ensemble_Members.size):
    wAVD_dST_CAM3Grid_AA[i,:] = Area_Avg.LatLonavg_Time(\
    wAVD_dST_CAM3Grid[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    wAVD_dST_CAM3Grid_GA[i,:] = Area_Avg.LatLonavg_Time(\
    wAVD_dST_CAM3Grid[i,:,:,:],CAM3_Lat,CAM3_Lon)

In [92]:
CAM3_wAVD_CLD_SW_Feedback_AA = CAM3_wAVD_CLD_SW_EnergyB_AA/wAVD_dST_CAM3Grid_AA
CAM3_wAVD_CLD_SW_Feedback_GA = CAM3_wAVD_CLD_SW_EnergyB_GA/wAVD_dST_CAM3Grid_GA
CAM3_wAVD_CLD_LW_Feedback_AA = CAM3_wAVD_CLD_LW_EnergyB_AA/wAVD_dST_CAM3Grid_AA
CAM3_wAVD_CLD_LW_Feedback_GA = CAM3_wAVD_CLD_LW_EnergyB_GA/wAVD_dST_CAM3Grid_GA

In [93]:
CAM3_wAVD_CLD_LW_EnergyB_Annual_AA = np.mean(CAM3_wAVD_CLD_LW_EnergyB_AA,axis=1)
CAM3_wAVD_CLD_LW_EnergyB_SON_AA = np.mean(CAM3_wAVD_CLD_LW_EnergyB_AA[:,8:11],axis=1)
CAM3_wAVD_CLD_LW_EnergyB_JJA_AA = np.mean(CAM3_wAVD_CLD_LW_EnergyB_AA[:,5:8],axis=1)
CAM3_wAVD_CLD_LW_EnergyB_MAM_AA = np.mean(CAM3_wAVD_CLD_LW_EnergyB_AA[:,2:5],axis=1)
CAM3_wAVD_CLD_LW_EnergyB_DJF_AA = np.mean((CAM3_wAVD_CLD_LW_EnergyB_AA[:,0],
                                       CAM3_wAVD_CLD_LW_EnergyB_AA[:,1],\
                                       CAM3_wAVD_CLD_LW_EnergyB_AA[:,11]),axis=0)

CAM3_wAVD_CLD_LW_EnergyB_Annual_GA = np.mean(CAM3_wAVD_CLD_LW_EnergyB_GA,axis=1)
CAM3_wAVD_CLD_LW_EnergyB_SON_GA = np.mean(CAM3_wAVD_CLD_LW_EnergyB_GA[:,8:11],axis=1)
CAM3_wAVD_CLD_LW_EnergyB_JJA_GA = np.mean(CAM3_wAVD_CLD_LW_EnergyB_GA[:,5:8],axis=1)
CAM3_wAVD_CLD_LW_EnergyB_MAM_GA = np.mean(CAM3_wAVD_CLD_LW_EnergyB_GA[:,2:5],axis=1)
CAM3_wAVD_CLD_LW_EnergyB_DJF_GA = np.mean((CAM3_wAVD_CLD_LW_EnergyB_GA[:,0],
                                       CAM3_wAVD_CLD_LW_EnergyB_GA[:,1],\
                                       CAM3_wAVD_CLD_LW_EnergyB_GA[:,11]),axis=0)

In [94]:
CAM3_wAVD_CLD_SW_EnergyB_Annual_AA = np.mean(CAM3_wAVD_CLD_SW_EnergyB_AA,axis=1)
CAM3_wAVD_CLD_SW_EnergyB_SON_AA = np.mean(CAM3_wAVD_CLD_SW_EnergyB_AA[:,8:11],axis=1)
CAM3_wAVD_CLD_SW_EnergyB_JJA_AA = np.mean(CAM3_wAVD_CLD_SW_EnergyB_AA[:,5:8],axis=1)
CAM3_wAVD_CLD_SW_EnergyB_MAM_AA = np.mean(CAM3_wAVD_CLD_SW_EnergyB_AA[:,2:5],axis=1)
CAM3_wAVD_CLD_SW_EnergyB_DJF_AA = np.mean((CAM3_wAVD_CLD_SW_EnergyB_AA[:,0],
                                       CAM3_wAVD_CLD_SW_EnergyB_AA[:,1],\
                                       CAM3_wAVD_CLD_SW_EnergyB_AA[:,11]),axis=0)

CAM3_wAVD_CLD_SW_EnergyB_Annual_GA = np.mean(CAM3_wAVD_CLD_SW_EnergyB_GA,axis=1)
CAM3_wAVD_CLD_SW_EnergyB_SON_GA = np.mean(CAM3_wAVD_CLD_SW_EnergyB_GA[:,8:11],axis=1)
CAM3_wAVD_CLD_SW_EnergyB_JJA_GA = np.mean(CAM3_wAVD_CLD_SW_EnergyB_GA[:,5:8],axis=1)
CAM3_wAVD_CLD_SW_EnergyB_MAM_GA = np.mean(CAM3_wAVD_CLD_SW_EnergyB_GA[:,2:5],axis=1)
CAM3_wAVD_CLD_SW_EnergyB_DJF_GA = np.mean((CAM3_wAVD_CLD_SW_EnergyB_GA[:,0],
                                       CAM3_wAVD_CLD_SW_EnergyB_GA[:,1],\
                                       CAM3_wAVD_CLD_SW_EnergyB_GA[:,11]),axis=0)

In [95]:
CAM3_wAVD_CLD_LW_Feedback_Annual_AA = np.mean(CAM3_wAVD_CLD_LW_Feedback_AA,axis=1)
CAM3_wAVD_CLD_LW_Feedback_SON_AA = np.mean(CAM3_wAVD_CLD_LW_Feedback_AA[:,8:11],axis=1)
CAM3_wAVD_CLD_LW_Feedback_JJA_AA = np.mean(CAM3_wAVD_CLD_LW_Feedback_AA[:,5:8],axis=1)
CAM3_wAVD_CLD_LW_Feedback_MAM_AA = np.mean(CAM3_wAVD_CLD_LW_Feedback_AA[:,2:5],axis=1)
CAM3_wAVD_CLD_LW_Feedback_DJF_AA = np.mean((CAM3_wAVD_CLD_LW_Feedback_AA[:,0],
                                       CAM3_wAVD_CLD_LW_Feedback_AA[:,1],\
                                       CAM3_wAVD_CLD_LW_Feedback_AA[:,11]),axis=0)

CAM3_wAVD_CLD_LW_Feedback_Annual_GA = np.mean(CAM3_wAVD_CLD_LW_Feedback_GA,axis=1)
CAM3_wAVD_CLD_LW_Feedback_SON_GA = np.mean(CAM3_wAVD_CLD_LW_Feedback_GA[:,8:11],axis=1)
CAM3_wAVD_CLD_LW_Feedback_JJA_GA = np.mean(CAM3_wAVD_CLD_LW_Feedback_GA[:,5:8],axis=1)
CAM3_wAVD_CLD_LW_Feedback_MAM_GA = np.mean(CAM3_wAVD_CLD_LW_Feedback_GA[:,2:5],axis=1)
CAM3_wAVD_CLD_LW_Feedback_DJF_GA = np.mean((CAM3_wAVD_CLD_LW_Feedback_GA[:,0],
                                       CAM3_wAVD_CLD_LW_Feedback_GA[:,1],\
                                       CAM3_wAVD_CLD_LW_Feedback_GA[:,11]),axis=0)

CAM3_wAVD_CLD_SW_Feedback_Annual_AA = np.mean(CAM3_wAVD_CLD_SW_Feedback_AA,axis=1)
CAM3_wAVD_CLD_SW_Feedback_SON_AA = np.mean(CAM3_wAVD_CLD_SW_Feedback_AA[:,8:11],axis=1)
CAM3_wAVD_CLD_SW_Feedback_JJA_AA = np.mean(CAM3_wAVD_CLD_SW_Feedback_AA[:,5:8],axis=1)
CAM3_wAVD_CLD_SW_Feedback_MAM_AA = np.mean(CAM3_wAVD_CLD_SW_Feedback_AA[:,2:5],axis=1)
CAM3_wAVD_CLD_SW_Feedback_DJF_AA = np.mean((CAM3_wAVD_CLD_SW_Feedback_AA[:,0],
                                       CAM3_wAVD_CLD_SW_Feedback_AA[:,1],\
                                       CAM3_wAVD_CLD_SW_Feedback_AA[:,11]),axis=0)

CAM3_wAVD_CLD_SW_Feedback_Annual_GA = np.mean(CAM3_wAVD_CLD_SW_Feedback_GA,axis=1)
CAM3_wAVD_CLD_SW_Feedback_SON_GA = np.mean(CAM3_wAVD_CLD_SW_Feedback_GA[:,8:11],axis=1)
CAM3_wAVD_CLD_SW_Feedback_JJA_GA = np.mean(CAM3_wAVD_CLD_SW_Feedback_GA[:,5:8],axis=1)
CAM3_wAVD_CLD_SW_Feedback_MAM_GA = np.mean(CAM3_wAVD_CLD_SW_Feedback_GA[:,2:5],axis=1)
CAM3_wAVD_CLD_SW_Feedback_DJF_GA = np.mean((CAM3_wAVD_CLD_SW_Feedback_GA[:,0],
                                       CAM3_wAVD_CLD_SW_Feedback_GA[:,1],\
                                       CAM3_wAVD_CLD_SW_Feedback_GA[:,11]),axis=0)

In [96]:
CAM3_wAVD_CLD_LW_EnergyB_Annual_AA_file = open("CAM3_wAVD_CLD_LW_EnergyB_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_Annual_AA,CAM3_wAVD_CLD_LW_EnergyB_Annual_AA_file)
CAM3_wAVD_CLD_LW_EnergyB_Annual_AA_file.close()

CAM3_wAVD_CLD_LW_EnergyB_SON_AA_file = open("CAM3_wAVD_CLD_LW_EnergyB_SON_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_SON_AA,CAM3_wAVD_CLD_LW_EnergyB_SON_AA_file)
CAM3_wAVD_CLD_LW_EnergyB_SON_AA_file.close()

CAM3_wAVD_CLD_LW_EnergyB_JJA_AA_file = open("CAM3_wAVD_CLD_LW_EnergyB_JJA_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_JJA_AA,CAM3_wAVD_CLD_LW_EnergyB_JJA_AA_file)
CAM3_wAVD_CLD_LW_EnergyB_JJA_AA_file.close()

CAM3_wAVD_CLD_LW_EnergyB_MAM_AA_file = open("CAM3_wAVD_CLD_LW_EnergyB_MAM_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_MAM_AA,CAM3_wAVD_CLD_LW_EnergyB_MAM_AA_file)
CAM3_wAVD_CLD_LW_EnergyB_MAM_AA_file.close()

CAM3_wAVD_CLD_LW_EnergyB_DJF_AA_file = open("CAM3_wAVD_CLD_LW_EnergyB_DJF_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_DJF_AA,CAM3_wAVD_CLD_LW_EnergyB_DJF_AA_file)
CAM3_wAVD_CLD_LW_EnergyB_DJF_AA_file.close()

CAM3_wAVD_CLD_LW_Feedback_Annual_AA_file = open("CAM3_wAVD_CLD_LW_Feedback_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_Annual_AA,CAM3_wAVD_CLD_LW_Feedback_Annual_AA_file)
CAM3_wAVD_CLD_LW_Feedback_Annual_AA_file.close()

CAM3_wAVD_CLD_LW_Feedback_SON_AA_file = open("CAM3_wAVD_CLD_LW_Feedback_SON_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_SON_AA,CAM3_wAVD_CLD_LW_Feedback_SON_AA_file)
CAM3_wAVD_CLD_LW_Feedback_SON_AA_file.close()

CAM3_wAVD_CLD_LW_Feedback_JJA_AA_file = open("CAM3_wAVD_CLD_LW_Feedback_JJA_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_JJA_AA,CAM3_wAVD_CLD_LW_Feedback_JJA_AA_file)
CAM3_wAVD_CLD_LW_Feedback_JJA_AA_file.close()

CAM3_wAVD_CLD_LW_Feedback_MAM_AA_file = open("CAM3_wAVD_CLD_LW_Feedback_MAM_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_MAM_AA,CAM3_wAVD_CLD_LW_Feedback_MAM_AA_file)
CAM3_wAVD_CLD_LW_Feedback_MAM_AA_file.close()

CAM3_wAVD_CLD_LW_Feedback_DJF_AA_file = open("CAM3_wAVD_CLD_LW_Feedback_DJF_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_DJF_AA,CAM3_wAVD_CLD_LW_Feedback_DJF_AA_file)
CAM3_wAVD_CLD_LW_Feedback_DJF_AA_file.close()

In [97]:
CAM3_wAVD_CLD_LW_EnergyB_Annual_GA_file = open("CAM3_wAVD_CLD_LW_EnergyB_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_Annual_GA,CAM3_wAVD_CLD_LW_EnergyB_Annual_GA_file)
CAM3_wAVD_CLD_LW_EnergyB_Annual_GA_file.close()

CAM3_wAVD_CLD_LW_EnergyB_SON_GA_file = open("CAM3_wAVD_CLD_LW_EnergyB_SON_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_SON_GA,CAM3_wAVD_CLD_LW_EnergyB_SON_GA_file)
CAM3_wAVD_CLD_LW_EnergyB_SON_GA_file.close()

CAM3_wAVD_CLD_LW_EnergyB_JJA_GA_file = open("CAM3_wAVD_CLD_LW_EnergyB_JJA_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_JJA_GA,CAM3_wAVD_CLD_LW_EnergyB_JJA_GA_file)
CAM3_wAVD_CLD_LW_EnergyB_JJA_GA_file.close()

CAM3_wAVD_CLD_LW_EnergyB_MAM_GA_file = open("CAM3_wAVD_CLD_LW_EnergyB_MAM_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_MAM_GA,CAM3_wAVD_CLD_LW_EnergyB_MAM_GA_file)
CAM3_wAVD_CLD_LW_EnergyB_MAM_GA_file.close()

CAM3_wAVD_CLD_LW_EnergyB_DJF_GA_file = open("CAM3_wAVD_CLD_LW_EnergyB_DJF_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_EnergyB_DJF_GA,CAM3_wAVD_CLD_LW_EnergyB_DJF_GA_file)
CAM3_wAVD_CLD_LW_EnergyB_DJF_GA_file.close()

CAM3_wAVD_CLD_LW_Feedback_Annual_GA_file = open("CAM3_wAVD_CLD_LW_Feedback_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_Annual_GA,CAM3_wAVD_CLD_LW_Feedback_Annual_GA_file)
CAM3_wAVD_CLD_LW_Feedback_Annual_GA_file.close()

CAM3_wAVD_CLD_LW_Feedback_SON_GA_file = open("CAM3_wAVD_CLD_LW_Feedback_SON_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_SON_GA,CAM3_wAVD_CLD_LW_Feedback_SON_GA_file)
CAM3_wAVD_CLD_LW_Feedback_SON_GA_file.close()

CAM3_wAVD_CLD_LW_Feedback_JJA_GA_file = open("CAM3_wAVD_CLD_LW_Feedback_JJA_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_JJA_GA,CAM3_wAVD_CLD_LW_Feedback_JJA_GA_file)
CAM3_wAVD_CLD_LW_Feedback_JJA_GA_file.close()

CAM3_wAVD_CLD_LW_Feedback_MAM_GA_file = open("CAM3_wAVD_CLD_LW_Feedback_MAM_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_MAM_GA,CAM3_wAVD_CLD_LW_Feedback_MAM_GA_file)
CAM3_wAVD_CLD_LW_Feedback_MAM_GA_file.close()

CAM3_wAVD_CLD_LW_Feedback_DJF_GA_file = open("CAM3_wAVD_CLD_LW_Feedback_DJF_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_LW_Feedback_DJF_GA,CAM3_wAVD_CLD_LW_Feedback_DJF_GA_file)
CAM3_wAVD_CLD_LW_Feedback_DJF_GA_file.close()

In [98]:
CAM3_wAVD_CLD_SW_EnergyB_Annual_AA_file = open("CAM3_wAVD_CLD_SW_EnergyB_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_Annual_AA,CAM3_wAVD_CLD_SW_EnergyB_Annual_AA_file)
CAM3_wAVD_CLD_SW_EnergyB_Annual_AA_file.close()

CAM3_wAVD_CLD_SW_EnergyB_SON_AA_file = open("CAM3_wAVD_CLD_SW_EnergyB_SON_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_SON_AA,CAM3_wAVD_CLD_SW_EnergyB_SON_AA_file)
CAM3_wAVD_CLD_SW_EnergyB_SON_AA_file.close()

CAM3_wAVD_CLD_SW_EnergyB_JJA_AA_file = open("CAM3_wAVD_CLD_SW_EnergyB_JJA_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_JJA_AA,CAM3_wAVD_CLD_SW_EnergyB_JJA_AA_file)
CAM3_wAVD_CLD_SW_EnergyB_JJA_AA_file.close()

CAM3_wAVD_CLD_SW_EnergyB_MAM_AA_file = open("CAM3_wAVD_CLD_SW_EnergyB_MAM_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_MAM_AA,CAM3_wAVD_CLD_SW_EnergyB_MAM_AA_file)
CAM3_wAVD_CLD_SW_EnergyB_MAM_AA_file.close()

CAM3_wAVD_CLD_SW_EnergyB_DJF_AA_file = open("CAM3_wAVD_CLD_SW_EnergyB_DJF_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_DJF_AA,CAM3_wAVD_CLD_SW_EnergyB_DJF_AA_file)
CAM3_wAVD_CLD_SW_EnergyB_DJF_AA_file.close()

CAM3_wAVD_CLD_SW_Feedback_Annual_AA_file = open("CAM3_wAVD_CLD_SW_Feedback_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_Annual_AA,CAM3_wAVD_CLD_SW_Feedback_Annual_AA_file)
CAM3_wAVD_CLD_SW_Feedback_Annual_AA_file.close()

CAM3_wAVD_CLD_SW_Feedback_SON_AA_file = open("CAM3_wAVD_CLD_SW_Feedback_SON_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_SON_AA,CAM3_wAVD_CLD_SW_Feedback_SON_AA_file)
CAM3_wAVD_CLD_SW_Feedback_SON_AA_file.close()

CAM3_wAVD_CLD_SW_Feedback_JJA_AA_file = open("CAM3_wAVD_CLD_SW_Feedback_JJA_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_JJA_AA,CAM3_wAVD_CLD_SW_Feedback_JJA_AA_file)
CAM3_wAVD_CLD_SW_Feedback_JJA_AA_file.close()

CAM3_wAVD_CLD_SW_Feedback_MAM_AA_file = open("CAM3_wAVD_CLD_SW_Feedback_MAM_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_MAM_AA,CAM3_wAVD_CLD_SW_Feedback_MAM_AA_file)
CAM3_wAVD_CLD_SW_Feedback_MAM_AA_file.close()

CAM3_wAVD_CLD_SW_Feedback_DJF_AA_file = open("CAM3_wAVD_CLD_SW_Feedback_DJF_AA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_DJF_AA,CAM3_wAVD_CLD_SW_Feedback_DJF_AA_file)
CAM3_wAVD_CLD_SW_Feedback_DJF_AA_file.close()

In [99]:
CAM3_wAVD_CLD_SW_EnergyB_Annual_GA_file = open("CAM3_wAVD_CLD_SW_EnergyB_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_Annual_GA,CAM3_wAVD_CLD_SW_EnergyB_Annual_GA_file)
CAM3_wAVD_CLD_SW_EnergyB_Annual_GA_file.close()

CAM3_wAVD_CLD_SW_EnergyB_SON_GA_file = open("CAM3_wAVD_CLD_SW_EnergyB_SON_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_SON_GA,CAM3_wAVD_CLD_SW_EnergyB_SON_GA_file)
CAM3_wAVD_CLD_SW_EnergyB_SON_GA_file.close()

CAM3_wAVD_CLD_SW_EnergyB_JJA_GA_file = open("CAM3_wAVD_CLD_SW_EnergyB_JJA_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_JJA_GA,CAM3_wAVD_CLD_SW_EnergyB_JJA_GA_file)
CAM3_wAVD_CLD_SW_EnergyB_JJA_GA_file.close()

CAM3_wAVD_CLD_SW_EnergyB_MAM_GA_file = open("CAM3_wAVD_CLD_SW_EnergyB_MAM_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_MAM_GA,CAM3_wAVD_CLD_SW_EnergyB_MAM_GA_file)
CAM3_wAVD_CLD_SW_EnergyB_MAM_GA_file.close()

CAM3_wAVD_CLD_SW_EnergyB_DJF_GA_file = open("CAM3_wAVD_CLD_SW_EnergyB_DJF_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_EnergyB_DJF_GA,CAM3_wAVD_CLD_SW_EnergyB_DJF_GA_file)
CAM3_wAVD_CLD_SW_EnergyB_DJF_GA_file.close()

CAM3_wAVD_CLD_SW_Feedback_Annual_GA_file = open("CAM3_wAVD_CLD_SW_Feedback_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_Annual_GA,CAM3_wAVD_CLD_SW_Feedback_Annual_GA_file)
CAM3_wAVD_CLD_SW_Feedback_Annual_GA_file.close()

CAM3_wAVD_CLD_SW_Feedback_SON_GA_file = open("CAM3_wAVD_CLD_SW_Feedback_SON_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_SON_GA,CAM3_wAVD_CLD_SW_Feedback_SON_GA_file)
CAM3_wAVD_CLD_SW_Feedback_SON_GA_file.close()

CAM3_wAVD_CLD_SW_Feedback_JJA_GA_file = open("CAM3_wAVD_CLD_SW_Feedback_JJA_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_JJA_GA,CAM3_wAVD_CLD_SW_Feedback_JJA_GA_file)
CAM3_wAVD_CLD_SW_Feedback_JJA_GA_file.close()

CAM3_wAVD_CLD_SW_Feedback_MAM_GA_file = open("CAM3_wAVD_CLD_SW_Feedback_MAM_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_MAM_GA,CAM3_wAVD_CLD_SW_Feedback_MAM_GA_file)
CAM3_wAVD_CLD_SW_Feedback_MAM_GA_file.close()

CAM3_wAVD_CLD_SW_Feedback_DJF_GA_file = open("CAM3_wAVD_CLD_SW_Feedback_DJF_GA.pickle","wb")
pickle.dump(CAM3_wAVD_CLD_SW_Feedback_DJF_GA,CAM3_wAVD_CLD_SW_Feedback_DJF_GA_file)
CAM3_wAVD_CLD_SW_Feedback_DJF_GA_file.close()